# Dataset Generation Parameters Glossary
**THIS IS A GLOSSARY MEANT TO BE A REFERENCE, THE CODE CELLS ARE NOT MEANT TO BE EXECUTED**

To build the dataset, we'll be working with three types of files that make up each sample:

- **Graph topology**: This file represents a graph topology, including the nodes and edges that forms it as well as characterstics of each.
- **Routing file**: This file shows the paths that traffic can take between each node within the graph topology.
- **Traffic matrix (TM)**: This file represents the flows of traffic through a given network. It describe the traffic moving from one node to another.

Each sample in the dataset is identified by a unique combination of these three files. This means that we can generate multiple samples from the same graph topology by pairing it with different traffic matrices, for example.

In the following sections, we'll show you how to generate these files, and how their properties can be altered to create different variations of the dataset.

In [1]:
import networkx as nx
import random

In [2]:
# Generate, for instance, a complete graph
G = nx.complete_graph(10)

# Set the number of ToS that the input traffic of the network can use. If it is not defined, the simulator sets it to 1.
G.graph["levelsToS"] = 3

# Assign bandwidth to each edge of the graph. Its value is considered in bps.
for (n0,n1) in G.edges():
    G[n0][n1]["bandwidth"] = 100000

Each node in our network simulation is defined by two key characteristics:

- **Scheduling policy**: The way in which packets are processed at each output port is determined by the state of the queues and the chosen scheduling policy. We consider the following four policies:
    - *First In First Out (FIFO)*: all packets are placed in a single queue and served in the order they arrived, regardless of their type of service (ToS).
    - *Strict Priority (SP)*: one queue is designated for each priority level, and packets in higher priority queues are served first.
    - *Weighted Fair Queueing (WFQ)*: each queue is assigned a weight according to the configuration. The sum of all weights must equal 100. The policy selects a queue to serve based on its weight and the data rate of that queue, ensuring fairness among all queues.
    - *Deficit Round Robin  (DRR)*: similar to WFQ, each queue is assigned a weight according to the configuration. The sum of all weights must equal 100. The policy cycles through the queues, dedicating time to each queue proportional to its weight.
- **Buffer size**: the amount of storage space available at each output port for packets that are waiting to be processed. When a packet arrives and the outgoing queue is full, it will be dropped. The buffer size is measured in bits, with a minimum value of 8000 bits.

It's important to remember that you will have to define these characteristics for all nodes in the topology. The scheduling policy of a node is stored in the attribute schedulingPolicy as a string.

Here's some examples of nodes with the scheduling policy and buffer size defined:


In [3]:
# Let's configure all the nodes with a FIFO policy
for node in G:
    G.nodes[node]["schedulingPolicy"] = "FIFO"

When it comes to simulating network traffic, it's important to be able to control how packets with different types of service (ToS) are handled within the network. With the scheduling policies SP, WFQ, and DRR, we have an additional level of control through the tosToQoSqueue attribute.

This attribute allows us to specify to which queues packets with different ToS should be placed in. For example, if we set the attribute to "0;1,2", we are telling the simulator to create two queues. Traffic with ToS 0 will be assigned to the first queue, and traffic with ToS 1 and 2 will be assigned to the second queue.

If the tosToQoSqueue attribute is not set, the simulator will use a default behavior: it will create one queue for each ToS defined in the levelsToS attribute, and assign one ToS to each queue.

Here's an example of a node with the tosToQoSqueue attribute defined:

In [ ]:
# Let's configure all the nodes with a SP policy with two queues.
for node in G:
    G.nodes[node]["schedulingPolicy"] = "SP"
    G.nodes[node]["tosToQoSqueue"] = "0;1,2"

When it comes to the scheduling policies of WFQ and DRR, we have an additional level of control over how packets are processed through the use of "weights" assigned to each queue. This is where the attribute schedulingWeights comes into play.

To use this attribute, we will feed it a string containing the weights for each queue, separated by commas. For example, if we have 3 queues and we want to assign weights of 45, 30 and 25 to them respectively, the attribute will be set as "45,30,25".

It is important to note that the sum of all weights must equal 100, otherwise the simulation will not work as expected.

Here's an example of a node with the schedulingWeights attribute defined:

In [8]:
# Let's configure all the nodes with a WFQ policy with three queues
for node in G:
    G.nodes[node]["schedulingPolicy"] = "WFQ"
    G.nodes[node]["tosToQoSqueue"] = "0;1;2"
    G.nodes[node]["schedulingWeights"] = "45, 30, 25"

In [ ]:
# Let's configure all the nodes with a DRR policy with three queues
for node in G:
    G.nodes[node]["schedulingPolicy"] = "DRR"
    G.nodes[node]["tosToQoSqueue"] = "0;1;2"
    G.nodes[node]["schedulingWeights"] = "45, 30, 25"

To configure the buffer size in our network simulation, we only need to modify the attribute bufferSizes and include the desired size of the buffer in bits. It's important to keep in mind that the buffer size should be greater than 8000 bits.

Here's an example of a node with the bufferSizes attribute defined:

In [4]:
# Assign to each node a queue size of 32000 bits
for node in G:
    G.nodes[node]["bufferSizes"] = 32000

In [5]:
# Finally we save the topology
graph_file = "graph.txt"
nx.write_gml(G,graph_file)

## Routing
The routing information for our network simulation is expressed in a text file, where each line represents a path through the network as a sequence of nodes. There are two types of routing that can be used: destination-based and source-destination-based routing.

Destination-based routing is where packets are forwarded to the next hop based solely on their destination address. In contrast, source-destination-based routing takes into account both the source and destination addresses when forwarding packets.

It's important to note that both types of routing should not contain loops. Loops can cause packets to circulate indefinitely, leading to network congestion and decreased performance.

Here's an example of a routing file using destination-based routing:

In [6]:
# For instance, we can use networkx to calculate the shortest path routing for each src-dst pair.
with open("routing.txt","w") as r_fd:
    lPaths = nx.shortest_path(G)
    for src in G:
        for dst in G:
            if src == dst:
                continue
            path =  ','.join(str(x) for x in lPaths[src][dst])
            r_fd.write(path+"\n")
        

## Traffic Matrix
The final step in generating our network simulation is to create the traffic matrix (TM) file. This file contains information on the different traffic flows that will be present in the network. Each line in the file describes one flow, and the parameters are separated by commas. Here's a breakdown of the parameters:

- **source and destination**: These parameters indicate the source and destination nodes for the given flow. 

- **avg_bw**: This parameter indicates the average bandwidth, in bps, to be generated for this flow. It's minimum value is 10 bps

- **time_distribution**: This parameter indicates how often packets should be generated over time. We support three time distributions: Poisson(```time_distribution```=1) , CBR(```time_distribution```=1) and ON-OFF(```time_distribution```=2). For Poisson and CBR, we do not need to define additional parameters, as the packets will be generated considering the chosen packet size distribution and average bandwidth parameters. For ON-OFF, we will need to define the length of the activity and inactivity periods (```on_time``` and ```off_time``` respectively). The on_time defines the average duration of the ON period. The duration of every ON period is modeled as an exponential distribution of average on_time seconds. The off_time follow the same principle as the the on_time.

- **pkt_dist**: This parameter notes the distribution type used to generate the packets. Currently, only one distribution is supported, so the value of pkt_dist should always be 0.

- **pkt_size_n** and **prob_n**: These parameters are used to indicate the possible sizes of the packets and their relative frequency within the flow. At least one packet size must be declared, but we can define up to 5 different sizes. The packet size should be a value between 50 and 2000 bits, while the sum of all the prob_n values should equal 1.

- **tos**: This parameter indicates the ToS assigned to the packets generated for this flow. When defining the tos to be used, select consecutive values starting from 0. For instance if you want to use 3 different ToS, select 0, 1 and 2.

At the end, the resulting line should look something like this:

```source, destination, avg_bw, time_distribution, [on_time, off_time,] pkt_dist, pkt_size_1, prob_1, [pkt_size_2, prob_2, [pkt_size_3, prob_3, [pkt_size_4, prob_4, [pkt_size_5, prob_5,]]]] tos```

*Note:* It's possible to define more than one flow between two nodes, although this functionality is not fully tested and it's up to the user to use it. However, keep in mind that as the simulator provides performance metrics aggregated per path and for each flow, the size of the dataset can increase easily.

By generating the traffic matrix file, you can control the different types of traffic that will be present in the network, leading to more accurate and realistic network traffic simulations.


In [7]:
"""
Example: this code will generate flows between all nodes in the graph, such as:
- The average bandwidth is randomized between 10 and 10000 bps
- An ON-OFF time distribution is used, with an average on_time of 5 s and an average off_time of 10 s
- Packets can have two possible sizes, 300 and 1700 bits, both equally probable
- The ToS for all flows is 0 (high priority)
"""
with open("traffic.txt","w") as tm_fd:
    for src in G:
        for dst in G:
            avg_bw = random.randint(10,10000)
            time_dist = 2
            on_time = 5
            off_time = 10
            pkt_size_1 = 300
            prob_1 = 0.5
            pkt_size_2 = 1700
            prob_2 = 0.5
            tos = 0
            traffic_line = "{},{},{},{},{},{},0,{},{},{},{},{}".format(
                src,dst,avg_bw,time_dist,on_time,off_time,pkt_size_1,
                prob_1,pkt_size_2,prob_2,tos)
            tm_fd.write(traffic_line+"\n")
